Nepal Earthquake OSM Data Exploration
=====================================

In [2]:
require '/opt/epic-osm/epic-osm.rb'
analysis_window = './nepal-earthquake.yml'
o  = EpicOSM.new(analysis_window: analysis_window)
aw = o.analysis_window

Successfully initialized Analysis Window: Nepal Earthquake



#<AnalysisWindow:0x007f5dde8b8d08 @bounding_box=#<BoundingBox:0x007f5dde8b8f88 @bbox_array=["80.0134", "26.3033", "88.2202", "30.5149"], @bottom_left=[80.0134, 26.3033], @top_right=[88.2202, 30.5149], @active=false>, @time_frame=#<TimeFrame:0x007f5dde8b94d8 @start_date=2015-04-25 00:00:00 +0000, @end_date=2015-05-31 00:00:00 +0000, @active=true>, @max_area=1000000000000, @min_area=1, @title="Nepal Earthquake", @changeset_tags=nil>

## Visualize some changesets information!

In [ ]:
#cSets_per_4hours = aw.changesets_x_hour(step: 4);
nil

In [ ]:
users_per_4hours = cSets_per_4hours.collect{|bucket| 
  {time: bucket[:start_date],
   all_users: bucket[:objects].collect{|x| x.user}.uniq,
   hot_users: bucket[:objects].select{|c| c.comment =~ /hot/}.collect{|x| x.user}.uniq
  }
}

counts_per_4_hours = users_per_4hours.collect{ |d| 
  { time:      d[:time],
    all_users: d[:all_users].count,
    hot_users: d[:hot_users].count
  }
}; nil

In [ ]:
puts "All users: #{ users_per_4hours.collect{|x| x[:all_users] }.flatten.uniq.length }"
puts "HOT users: #{ users_per_4hours.collect{|x| x[:hot_users] }.flatten.uniq.length }"
nil

In [ ]:
#Write out JSON to visualize:
require 'json'
File.write('json/user_counts_per_4hour.json', counts_per_4_hours.to_json)

## Calculate Cumulative New Users?

In [ ]:
all_users = Set.new
hot_users = Set.new
cumulative_users = []

cSets_per_4hours.each do |bucket|
  t_all_users = bucket[:objects].collect{|x| x.user}.uniq.to_set
  t_hot_users = bucket[:objects].select{|c| c.comment =~ /hot/}.collect{|x| x.user}.uniq.to_set
  
  
  #Unless THIS set of users is a subset of ALL PREVIOUS users... add the difference!
  unless t_all_users.subset?(all_users)
    all_users = all_users.merge(t_all_users - all_users)
  end
  
  unless t_hot_users.subset?(hot_users)
    hot_users = hot_users.merge(t_hot_users - hot_users)
  end
    
  cumulative_users << {time: bucket[:start_date], all_users: all_users.size, hot_users: hot_users.size}
  
end

cumulative_users[0..10].each do |u|
  puts "#{u[:time]}: #{u[:hot_users]}, #{u[:all_users]}"
end

File.write('json/cumulative_user_count.json', cumulative_users.to_json)
nil;

testing: {{x}}

## Building Statistics

In [ ]:
# 1. Get the buildings grouped by 4 hours:
buildings_per_4_hours = aw.ways_x_hour(step: 4, constraints: {version: 1, 'tags.building' => {'$ne' => nil}}); nil


In [ ]:
# 2. Sort the buildings
building_counts_per_4_hours = buildings_per_4_hours.collect{ |bucket| 
  by_users = bucket[:objects].group_by{|w| w.user }
  { 
    t: bucket[:start_date],
    c: bucket[:objects].size,
    u_count: by_users.size,
    per_user_count: by_users.collect{|k,v| {k => v.size}}
  }
}; nil;

In [ ]:
# 3. Write the JSON file.
require 'json'
# File.write('json/new_buildings.json', building_counts_per_4_hours.to_json)

In [ ]:
puts "There were #{building_counts_per_4_hours.collect{|b| b[:c]}.inject(:+)} new buildings added"

In [ ]:
uniq_users = building_counts_per_4_hours.collect{|b| b[:per_user_count].flatten.collect{|x| x.keys}}.flatten.uniq
uniq_users.size

In [ ]:
buildings_per_user = building_counts_per_4_hours.collect{|b| b[:per_user_count].flatten.group_by{|x| x.keys}}

## Road Statistics
Pull info on (new) roads!

In [ ]:
roads_per_4hour = aw.ways_x_hour(step: 4, constraints: {version: 1, 'tags.highway' => {'$ne' => nil}})

In [7]:
# Filter, Count, & Export
road_counts_per_4_hours = roads_per_4hour.collect{ |bucket| 
  by_users = bucket[:objects].group_by{|w| w.user }
  { 
    t: bucket[:start_date],
    c: bucket[:objects].size,
    u_count: by_users.size,
    per_user_count: by_users.collect{|k,v| {k => v.size}}
  }
}

require 'json'
File.write('json/new_roads.json', road_counts_per_4_hours.to_json)

210885

In [8]:
puts "There were #{road_counts_per_4_hours.collect{|b| b[:c]}.inject(:+)} new roads added"

There were 150097 new roads added
